# Data Wrangling in Pandas

This session draws primarily on Chapter 7 in Python for Data Analysis.  It covers methods that are used heavily in 'data wrangling', which refers to the data manipulation that is often needed to transform raw data into a form that is useful for analysis.  We'll stick to the data and examples used in the book for most of this session, since the examples are clearer on the tiny datasets.  After that we will work through some of these methods again using real data.

Key methods covered include:

* Merging and Concatenating
* Reshaping data
* Data transformations
* Categorization
* Detecting and Filtering Outliers
* Creating Dummy Variables


In [1]:
import pandas as pd
import numpy as np

## Merging

Merging two datasets is a very common operation in preparing data for analysis.  It generally means adding columns from one table to colums from another, where the value of some key, or merge field, matches.

Let's begin by creating two simple DataFrames to be merged.

In [2]:
df1 = pd.DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'a', 'b'],'data1': range(7)})
df2 = pd.DataFrame({'key': ['a', 'b', 'd'],'data2': range(3)})
print(df1)
print(df2)

  key  data1
0   b      0
1   b      1
2   a      2
3   c      3
4   a      4
5   a      5
6   b      6
  key  data2
0   a      0
1   b      1
2   d      2


Here is a many to one merge.  The join field is implicit, based on what columns it finds in common between the two dataframes. Note that they share some values of the key field (a, b), but do not share key values c and d.  What do you expect to happen when we merge them? The result contains the values from both inputs where they both have a value of the merge field, which is 'key' in this example.  The default behavior is that the key value has to be in both inputs to be kept.  In set terms it would be an intersection of the two sets.

In [3]:
pd.merge(df1,df2)

,key,data1,data2
0,b,0,1
1,b,1,1
2,b,6,1
3,a,2,0
4,a,4,0
5,a,5,0


Here is the same merge, but making the join field explicit.


In [4]:
pd.merge(df1,df2, on='key')

,key,data1,data2
0,b,0,1
1,b,1,1
2,b,6,1
3,a,2,0
4,a,4,0
5,a,5,0


In [5]:
#what if there are more than one value of key in both dataframes? This is a many-to-many merge.
df1 = pd.DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'a', 'b'],'data1': range(7)})
df3 = pd.DataFrame({'key': ['a', 'b', 'b', 'd'],'data2': range(4)})
print(df1)
print(df3)
pd.merge(df1,df3, on='key')
#This produces a cartesian product of the number of occurrences of each key value in both dataframes:
# (b shows up 3 times in df1 and 2 times in df3, so we get 6 occurrences in the result of the merge)

  key  data1
0   b      0
1   b      1
2   a      2
3   c      3
4   a      4
5   a      5
6   b      6
  key  data2
0   a      0
1   b      1
2   b      2
3   d      3


,key,data1,data2
0,b,0,1
1,b,0,2
2,b,1,1
3,b,1,2
4,b,6,1
5,b,6,2
6,a,2,0
7,a,4,0
8,a,5,0


In [6]:
# There are several types of joins: left, right, inner, and outer. Let's compare them.
# How does a 'left' join compare to our initial join?  Note that it keeps the result if it shows up in df1,
# regardless of whether it also shows up in df2.  It fills in a value of NaN for the missing value from df2.
pd.merge(df1,df3, on='key', how='left')

,key,data1,data2
0,b,0,1.0
1,b,0,2.0
2,b,1,1.0
3,b,1,2.0
4,a,2,0.0
5,c,3,NaN
6,a,4,0.0
7,a,5,0.0
8,b,6,1.0
9,b,6,2.0


In [7]:
#How does a 'right' join compare?  Same idea, but this time it keeps a result if it shows up in df2, regardless
# of whether it also shows up in df1.
pd.merge(df1,df3, on='key', how='right')

,key,data1,data2
0,b,0.0,1
1,b,1.0,1
2,b,6.0,1
3,b,0.0,2
4,b,1.0,2
5,b,6.0,2
6,a,2.0,0
7,a,4.0,0
8,a,5.0,0
9,d,NaN,3


In [8]:
#How does an 'inner' join compare?
pd.merge(df1,df3, on='key', how='inner')
# seems to be the default argument...

,key,data1,data2
0,b,0,1
1,b,0,2
2,b,1,1
3,b,1,2
4,b,6,1
5,b,6,2
6,a,2,0
7,a,4,0
8,a,5,0


In [9]:
#How does an 'outer' join compare?  If inner joins are like an intersection of two sets, outer joins are unions.
pd.merge(df1,df3, on='key', how='outer')

,key,data1,data2
0,b,0.0,1.0
1,b,0.0,2.0
2,b,1.0,1.0
3,b,1.0,2.0
4,b,6.0,1.0
5,b,6.0,2.0
6,a,2.0,0.0
7,a,4.0,0.0
8,a,5.0,0.0
9,c,3.0,NaN


In [10]:
#What if the join fields have different names?  No problem - just specify the names.
df4 = pd.DataFrame({'key_1': ['b', 'b', 'a', 'c', 'a', 'a', 'b'],'data1': range(7)})
df5 = pd.DataFrame({'key_2': ['a', 'b', 'b', 'd'],'data2': range(4)})
pd.merge(df4,df5, left_on='key_1', right_on='key_2')

,key_1,data1,key_2,data2
0,b,0,b,1
1,b,0,b,2
2,b,1,b,1
3,b,1,b,2
4,b,6,b,1
5,b,6,b,2
6,a,2,a,0
7,a,4,a,0
8,a,5,a,0


In [11]:
# Here is an example that uses a combination of a data column and an index to merge two dataframes.
df4 = pd.DataFrame({'key_1': ['b', 'b', 'a', 'c', 'a', 'a', 'b'],'data1': range(7)})
df5 = pd.DataFrame({'data2': [4,6,8,10]}, index=['a','b','c','d'])
pd.merge(df4,df5, left_on='key_1', right_index=True)

,key_1,data1,data2
0,b,0,6
1,b,1,6
6,b,6,6
2,a,2,4
4,a,4,4
5,a,5,4
3,c,3,8


## Concatenating

In [12]:
# Concatenating can append rows, or columns, depending on which axis you use. Default is 0
s1 = pd.Series([0, 1], index=['a', 'b'])
s2 = pd.Series([2, 3, 4], index=['c', 'd', 'e'])
s3 = pd.Series([5, 6], index=['f', 'g'])
pd.concat([s1, s2, s3])
# Since we are concatenating series on axis 0, this creates a longer series, appending each of the three series

a    0
b    1
c    2
d    3
e    4
f    5
g    6
dtype: int64

In [13]:
# What if we concatenate on axis 1?
pd.concat([s1, s2, s3], axis=1)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


,0,1,2
a,0.0,NaN,NaN
b,1.0,NaN,NaN
c,NaN,2.0,NaN
d,NaN,3.0,NaN
e,NaN,4.0,NaN
f,NaN,NaN,5.0
g,NaN,NaN,6.0


In [14]:
# Outer join is the default:
pd.concat([s1, s2, s3], axis=1, join='outer')

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


,0,1,2
a,0.0,NaN,NaN
b,1.0,NaN,NaN
c,NaN,2.0,NaN
d,NaN,3.0,NaN
e,NaN,4.0,NaN
f,NaN,NaN,5.0
g,NaN,NaN,6.0


In [15]:
# What would an inner join produce?
pd.concat([s1, s2, s3], axis=1, join='inner')

,0,1,2


In [16]:
# We need some overlapping values to have the inner join produe non-empty results
s4 = pd.Series([4, 5, 6], index=['c', 'd', 'e'])
s5 = pd.Series([1, 2, 3], index=['d', 'e', 'f'])
s6 = pd.Series([7, 8, 9, 10], index=['d', 'e', 'f', 'g'])
pd.concat([s4, s5, s6], axis=1, join='outer')

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """


,0,1,2
c,4.0,NaN,NaN
d,5.0,1.0,7.0
e,6.0,2.0,8.0
f,NaN,3.0,9.0
g,NaN,NaN,10.0


In [17]:
# Here is the inner join 
pd.concat([s4, s5, s6], axis=1, join='inner')
# Note that it contains only entries that overlap in all three series.

,0,1,2
d,5,1,7
e,6,2,8


## Reshaping with Hierarchical Indexing

In [18]:
data = pd.DataFrame(np.arange(6).reshape((2, 3)),
                 index=pd.Index(['Ohio', 'Colorado'], name='state'),
                 columns=pd.Index(['one', 'two', 'three'], name='number'))
data

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


In [19]:
# Stack pivots the columns into rows, producing a Series with a hierarchical index:
result = data.stack()
result

state     number
Ohio      one       0
          two       1
          three     2
Colorado  one       3
          two       4
          three     5
dtype: int32

In [20]:
# Unstack reverses this process:
result.unstack()

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


See also the related pivot method

## Data Transformations

In [21]:
# Start with a dataframe containing some duplicate values
data = pd.DataFrame({'k1': ['one'] * 3 + ['two'] * 4,'k2': [1, 1, 2, 3, 3, 4, 99]})
data

,k1,k2
0,one,1
1,one,1
2,one,2
3,two,3
4,two,3
5,two,4
6,two,99


In [22]:
# How to see which rows contain duplicate values
data.duplicated()

0    False
1     True
2    False
3    False
4     True
5    False
6    False
dtype: bool

In [23]:
# How to remove duplicate values
data.drop_duplicates()

,k1,k2
0,one,1
2,one,2
3,two,3
5,two,4
6,two,99


In [24]:
#If 99 is a code for missing data, we could replace any such values with NaNs
data['k2'].replace(99,np.nan)

0    1.0
1    1.0
2    2.0
3    3.0
4    3.0
5    4.0
6    NaN
Name: k2, dtype: float64

## Categorization (binning)

In [25]:
# Let's look at how to create categories of data using ranges to bin the data using cut
ages = [20, 22, 25, 27, 21, 23, 37, 31, 61, 45, 41, 32]
bins = [18, 25, 35, 60, 100]
cats = pd.cut(ages, bins)
type(cats)

pandas.core.arrays.categorical.Categorical

In [26]:
cats.categories

IntervalIndex([(18, 25], (25, 35], (35, 60], (60, 100]]
              closed='right',
              dtype='interval[int64]')

In [27]:
cats.codes

array([0, 0, 0, 1, 0, 0, 2, 1, 3, 2, 2, 1], dtype=int8)

In [28]:
pd.value_counts(cats)

(18, 25]     5
(35, 60]     3
(25, 35]     3
(60, 100]    1
dtype: int64

In [29]:
# Consistent with mathematical notation for intervals, a parenthesis means that the side is open while the 
#square bracket means it is closed (inclusive). Which side is closed can be changed by passing right=False:
cats = pd.cut(ages, bins, right=False)
print(ages)
print(pd.value_counts(cats))

[20, 22, 25, 27, 21, 23, 37, 31, 61, 45, 41, 32]
[25, 35)     4
[18, 25)     4
[35, 60)     3
[60, 100)    1
dtype: int64


### Removing Outliers

In [30]:
# Start by creating a dataframe with 4 columns of 1,000 random numbers
# We'll use a fixed seed for the random number generator to get repeatable results
np.random.seed(12345)
data = pd.DataFrame(np.random.randn(1000, 4))
data.describe()

,0,1,2,3
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,-0.067684,0.067924,0.025598,-0.002298
std,0.998035,0.992106,1.006835,0.996794
min,-3.428254,-3.548824,-3.184377,-3.745356
25%,-0.774890,-0.591841,-0.641675,-0.644144
50%,-0.116401,0.101143,0.002073,-0.013611
75%,0.616366,0.780282,0.680391,0.654328
max,3.366626,2.653656,3.260383,3.927528


In [31]:
# This identifies any values in column 3 with absolute values > 3
col = data[3]
col[np.abs(col) > 3]

97     3.927528
305   -3.399312
400   -3.745356
Name: 3, dtype: float64

In [32]:
# This identifies all the rows with any column containing absolute values > 3
data[(np.abs(data) > 3).any(1)]

,0,1,2,3
5,-0.539741,0.476985,3.248944,-1.021228
97,-0.774363,0.552936,0.106061,3.927528
102,-0.655054,-0.565230,3.176873,0.959533
305,-2.315555,0.457246,-0.025907,-3.399312
324,0.050188,1.951312,3.260383,0.963301
400,0.146326,0.508391,-0.196713,-3.745356
499,-0.293333,-0.242459,-3.056990,1.918403
523,-3.428254,-0.296336,-0.439938,-0.867165
586,0.275144,1.179227,-3.184377,1.369891
808,-0.362528,-3.548824,1.553205,-2.186301


In [33]:
# Now we can cap the values at -3 to 3 using this:
data[np.abs(data) > 3] = np.sign(data) * 3
data.describe()

,0,1,2,3
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,-0.067623,0.068473,0.025153,-0.002081
std,0.995485,0.990253,1.003977,0.989736
min,-3.000000,-3.000000,-3.000000,-3.000000
25%,-0.774890,-0.591841,-0.641675,-0.644144
50%,-0.116401,0.101143,0.002073,-0.013611
75%,0.616366,0.780282,0.680391,0.654328
max,3.000000,2.653656,3.000000,3.000000


### Computing Dummy Variables

In [34]:
df = pd.DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'b'],'data1': range(6)})
df

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,b,5


In [35]:
# This generates dummy variables for each value of key
# Dummy variables are useful in statistical modeling, to have 0/1 indicator
# variables for the presence of some condition
pd.get_dummies(df['key'])

,a,b,c
0,0,1,0
1,0,1,0
2,1,0,0
3,0,0,1
4,1,0,0
5,0,1,0


In [36]:
# This generates dummy variables for each value of key and appends these to the dataframe
dummies = pd.get_dummies(df['key'], prefix='key')
df_with_dummy = df[['data1']].join(dummies)
df_with_dummy

,data1,key_a,key_b,key_c
0,0,0,1,0
1,1,0,1,0
2,2,1,0,0
3,3,0,0,1
4,4,1,0,0
5,5,0,1,0


Notice that we used join instead of merge.  The join method is very similar to merge, but uses indexes to merge, by default.  From the documentation:

http://pandas.pydata.org/pandas-docs/stable/merging.html#database-style-dataframe-joining-merging
merge is a function in the pandas namespace, and it is also available as a DataFrame instance method, with the calling DataFrame being implicitly considered the left object in the join.

The related DataFrame.join method, uses merge internally for the index-on-index and index-on-column(s) joins, but joins on indexes by default rather than trying to join on common columns (the default behavior for merge). If you are joining on index, you may wish to use DataFrame.join to save yourself some typing

## A bit more: 
1. Filter out records with more than 4 bedrooms
2. Create dummy variables for each bedroom count (e.g. bed_1 would have 1 for rows with 1 bedroom, 0 for others), and merge them with the dataframe
3. Filter sqft < 500 and > 3000
4. Create a set of 5 bins for price and do counts of how many records are in each category

Problem 1

In [37]:
items = pd.read_csv(r'C:\Users\Ryan Renfro\Documents\Spring 2019\CE 599\ce599\06-Pandas Basics Part 2/items.csv')
items

,neighborhood,title,price,bedrooms,pid,longitude,date,link,latitude,sqft,sourcepage
0,(SOMA / south beach),"1bed + Den, 1bath at Mission Bay",$2895,/ 1br - 950ft² -,4046628359,-122.399663,Sep 4 2013,/sfc/apa/4046628359.html,37.774623,/ 1br - 950ft² -,http://sfbay.craigslist.org/sfc/apa/
1,(SOMA / south beach),Love where you live!,$3354,/ 1br - 710ft² -,4046761563,NaN,Sep 4 2013,/sfc/apa/4046761563.html,NaN,/ 1br - 710ft² -,http://sfbay.craigslist.org/sfc/apa/
2,(inner sunset / UCSF),We Welcome Your Furry Friends! Call Today!,$2865,/ 1br - 644ft² -,4046661504,-122.470727,Sep 4 2013,/sfc/apa/4046661504.html,37.765739,/ 1br - 644ft² -,http://sfbay.craigslist.org/sfc/apa/
3,(financial district),Golden Gateway Commons | 2BR + office townhous...,$5500,/ 2br - 1450ft² -,4036170429,NaN,Sep 4 2013,/sfc/apa/4036170429.html,NaN,/ 2br - 1450ft² -,http://sfbay.craigslist.org/sfc/apa/
4,(lower nob hill),Experience Luxury Living in San Fransisco!,$3892,/ 2br -,4046732678,NaN,Sep 4 2013,/sfc/apa/4046732678.html,NaN,/ 2br -,http://sfbay.craigslist.org/sfc/apa/
5,(sunset / parkside),"$1250 - 1 bdrm, 1 bath",$1250,/ 1br -,4046731229,NaN,Sep 4 2013,/sfc/apa/4046731229.html,NaN,/ 1br -,http://sfbay.craigslist.org/sfc/apa/
6,(SOMA / south beach),Made For The Die-hard Giants Fan,$3249,/ 549ft² -,4046730047,-122.389798,Sep 4 2013,/sfc/apa/4046730047.html,37.774192,/ 549ft² -,http://sfbay.craigslist.org/sfc/apa/
7,(russian hill),Open Concept 1bed 1bath,$2690,/ 1br - 781ft² -,4046570245,-122.420787,Sep 4 2013,/sfc/apa/4046570245.html,37.796034,/ 1br - 781ft² -,http://sfbay.craigslist.org/sfc/apa/
8,NaN,"Contemporary, charming 2bds/1ba with private d...",$2850,/ 2br -,4006732632,-122.457100,Sep 4 2013,/sfc/apa/4006732632.html,37.735400,/ 2br -,http://sfbay.craigslist.org/sfc/apa/
9,(pacific heights),"2bd/2.5ba, 2 car tandem parking @ 1998 Broadwa...",$6500,/ 2br - 1400ft² -,4046018830,-122.429850,Sep 4 2013,/sfc/apa/4046018830.html,37.794973,/ 2br - 1400ft² -,http://sfbay.craigslist.org/sfc/apa/index200.html


In [38]:
#Clean bedroom data
cl = items
def clean_br(value):
    if isinstance(value, str):
        end = value.find('br')
        if end == -1:
            return None
        else:
            start = value.find('/') + 2
            return int(value[start:end])
cl['bedrooms'] = cl['bedrooms'].map(clean_br)
cl['bedrooms'][:9]

0    1.0
1    1.0
2    1.0
3    2.0
4    2.0
5    1.0
6    NaN
7    1.0
8    2.0
Name: bedrooms, dtype: float64

In [39]:
#Filter out records of more than 4 bedrooms
items_bed_filtered = items[items['bedrooms']<=4]

Problem 2

In [40]:
#Create dummy variables for bedrooms and merge to dataframe
dummies = pd.get_dummies(items_bed_filtered['bedrooms'], prefix='bed')
items_with_dummy = items_bed_filtered.join(dummies)
items_with_dummy

,neighborhood,title,price,bedrooms,pid,longitude,date,link,latitude,sqft,sourcepage,bed_1.0,bed_2.0,bed_3.0,bed_4.0
0,(SOMA / south beach),"1bed + Den, 1bath at Mission Bay",$2895,1.0,4046628359,-122.399663,Sep 4 2013,/sfc/apa/4046628359.html,37.774623,/ 1br - 950ft² -,http://sfbay.craigslist.org/sfc/apa/,1,0,0,0
1,(SOMA / south beach),Love where you live!,$3354,1.0,4046761563,NaN,Sep 4 2013,/sfc/apa/4046761563.html,NaN,/ 1br - 710ft² -,http://sfbay.craigslist.org/sfc/apa/,1,0,0,0
2,(inner sunset / UCSF),We Welcome Your Furry Friends! Call Today!,$2865,1.0,4046661504,-122.470727,Sep 4 2013,/sfc/apa/4046661504.html,37.765739,/ 1br - 644ft² -,http://sfbay.craigslist.org/sfc/apa/,1,0,0,0
3,(financial district),Golden Gateway Commons | 2BR + office townhous...,$5500,2.0,4036170429,NaN,Sep 4 2013,/sfc/apa/4036170429.html,NaN,/ 2br - 1450ft² -,http://sfbay.craigslist.org/sfc/apa/,0,1,0,0
4,(lower nob hill),Experience Luxury Living in San Fransisco!,$3892,2.0,4046732678,NaN,Sep 4 2013,/sfc/apa/4046732678.html,NaN,/ 2br -,http://sfbay.craigslist.org/sfc/apa/,0,1,0,0
5,(sunset / parkside),"$1250 - 1 bdrm, 1 bath",$1250,1.0,4046731229,NaN,Sep 4 2013,/sfc/apa/4046731229.html,NaN,/ 1br -,http://sfbay.craigslist.org/sfc/apa/,1,0,0,0
7,(russian hill),Open Concept 1bed 1bath,$2690,1.0,4046570245,-122.420787,Sep 4 2013,/sfc/apa/4046570245.html,37.796034,/ 1br - 781ft² -,http://sfbay.craigslist.org/sfc/apa/,1,0,0,0
8,NaN,"Contemporary, charming 2bds/1ba with private d...",$2850,2.0,4006732632,-122.457100,Sep 4 2013,/sfc/apa/4006732632.html,37.735400,/ 2br -,http://sfbay.craigslist.org/sfc/apa/,0,1,0,0
9,(pacific heights),"2bd/2.5ba, 2 car tandem parking @ 1998 Broadwa...",$6500,2.0,4046018830,-122.429850,Sep 4 2013,/sfc/apa/4046018830.html,37.794973,/ 2br - 1400ft² -,http://sfbay.craigslist.org/sfc/apa/index200.html,0,1,0,0
10,(SOMA / south beach),"Stunning Modern GARDEN Loft! Upgrades, Views &...",$4100,1.0,4045981009,-122.402387,Sep 4 2013,/sfc/apa/4045981009.html,37.781055,/ 1br -,http://sfbay.craigslist.org/sfc/apa/index200.html,1,0,0,0


Problem 3

In [41]:
#Clean sqft 


C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


0        / 1br - 950ft² -    
1        / 1br - 710ft² -    
2        / 1br - 644ft² -    
3       / 2br - 1450ft² -    
4                 / 2br -    
5                 / 1br -    
6              / 549ft² -    
7        / 1br - 781ft² -    
8                 / 2br -    
Name: sqft, dtype: object

In [42]:
#Filter sqft less than 500 and greater than 3000
items_sqft_filtered = items_bed_filtered[items_bed_filtered['sqft']>=500 and <=3000]

SyntaxError: invalid syntax (<ipython-input-42-c8cb0eed337c>, line 1)

Problem 4

In [45]:
#Create 5 bins for price
#Use qcut

ValueError: Input array must be 1 dimensional